## ローカルのデータ準備

### Download Data

In [1]:
import os
import requests
from zipfile import ZipFile

os.makedirs('./data', exist_ok=True)

url = 'https://github.com/asherif844/MLOps/raw/master/data/AdventureWorks-oltp-install-script.zip'
zip_data = requests.get(url)

with open('./data/adventureworks.zip', 'wb') as f:
    f.write(zip_data.content)

with ZipFile('./data/adventureworks.zip', 'r') as fzip:
    fzip.extractall('./data/csv_data')

### Transform Data

In [2]:
import pandas as pd

header = ['TransactionID', 'ProductID', 'ReferenceOrderID', 'ReferenceOrderLineID', 'TransactionDate', 'TransactionType', 'Quantity', 'ActualCost', 'ModifiedDate']

trans_hist_df = pd.read_csv('./data/csv_data/TransactionHistory.csv', sep='\t', names=header)

trans_hist_df['PaidAmount'] = trans_hist_df['Quantity'] * trans_hist_df['ActualCost']
trans_hist_df['TransactionDate'] = pd.to_datetime(trans_hist_df['TransactionDate'])
df = trans_hist_df[['TransactionDate', 'PaidAmount']]
df.set_index('TransactionDate',inplace=True)

df = df.resample('D').mean().interpolate()
df = df['2013-07':'2014-05']
df1 = df['2013']
df2 = df['2014']

df.to_csv('./data/mlops_forecast_data.csv', index=True, header=True)
df1.to_csv('./data/mlops_forecast_data2013.csv', index=True, header=True)
df2.to_csv('./data/mlops_forecast_data2014.csv', index=True, header=True)

## AML へのアップロード

### Azure へのログイン

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

### AML ワークスペースの環境設定

In [3]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

### transaction_ts のアップロード

In [4]:
%%writefile data_transaction_ts.yml
$schema: https://azuremlschemas.azureedge.net/latest/data.schema.json
name: transaction_ts
description: time series dataset for mlops
type: uri_file
path: data/mlops_forecast_data.csv

Writing data_transaction_ts.yml


In [5]:
!az ml data create --file data_transaction_ts.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml data' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Uploading mlops_forecast_data.csv (< 1 MB): 100%|█| 9.01k/9.01k [00:00<00:00, 75


{
  "creation_context": {
    "created_at": "2022-04-26T05:02:33.553359+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-04-26T05:02:33.562347+00:00"
  },
  "description": "time series dataset for mlops",
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/data/transaction_ts/versions/1",
  "name": "transaction_ts",
  "path": "azureml://subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/TEST2020426-ML/workspaces/ws01/datastores/workspaceblobstore/paths/LocalUpload/6280d99e622831eb8dfcc384fec22ce9/mlops_forecast_data.csv",
  "resourceGroup": "TEST2020426-ML",
  "tags": {},
  "type": "uri_file",
  "version": "1"
}


### transaction_ts2013 のアップロード

In [6]:
%%writefile transaction_ts2013.yml
$schema: https://azuremlschemas.azureedge.net/latest/data.schema.json
name: transaction_ts2013
description: 2013 time series dataset for mlops
type: uri_file
path: data/mlops_forecast_data2013.csv

Writing transaction_ts2013.yml


In [7]:
!az ml data create --file transaction_ts2013.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml data' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Uploading mlops_forecast_data2013.csv (< 1 MB): 100%|█| 4.57k/4.57k [00:00<00:00


{
  "creation_context": {
    "created_at": "2022-04-26T05:03:01.699074+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-04-26T05:03:01.708786+00:00"
  },
  "description": "2013 time series dataset for mlops",
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/data/transaction_ts2013/versions/1",
  "name": "transaction_ts2013",
  "path": "azureml://subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/TEST2020426-ML/workspaces/ws01/datastores/workspaceblobstore/paths/LocalUpload/c6da384a6036134c6bdcef21291509f7/mlops_forecast_data2013.csv",
  "resourceGroup": "TEST2020426-ML",
  "tags": {},
  "type": "uri_file",
  "ve

### transaction_ts2014 のアップロード

In [8]:
%%writefile transaction_ts2014.yml
$schema: https://azuremlschemas.azureedge.net/latest/data.schema.json
name: transaction_ts2014
description: 2014 time series dataset for mlops
type: uri_file
path: data/mlops_forecast_data2014.csv

Writing transaction_ts2014.yml


In [9]:
!az ml data create --file transaction_ts2014.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml data' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Uploading mlops_forecast_data2014.csv (< 1 MB): 100%|█| 4.47k/4.47k [00:00<00:00


{
  "creation_context": {
    "created_at": "2022-04-26T05:03:14.150629+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-04-26T05:03:14.162923+00:00"
  },
  "description": "2014 time series dataset for mlops",
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST2020426-ML/providers/Microsoft.MachineLearningServices/workspaces/ws01/data/transaction_ts2014/versions/1",
  "name": "transaction_ts2014",
  "path": "azureml://subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/TEST2020426-ML/workspaces/ws01/datastores/workspaceblobstore/paths/LocalUpload/a976d72fde03d487303fc1be33cfa266/mlops_forecast_data2014.csv",
  "resourceGroup": "TEST2020426-ML",
  "tags": {},
  "type": "uri_file",
  "ve